# Procesamiento natural del lenguaje
## Carrera de especialización en inteligencia artificial

# Desafío 2: 

## Embeddings de palabras basado en **Martin Fierro**
**Alumno:**

**Nahuel Otonelo Canale**

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from gensim.models.callbacks import CallbackAny2Vec

from sklearn.decomposition import IncrementalPCA
from sklearn.manifold import TSNE
import numpy as np

from gensim.models import Word2Vec

import plotly.graph_objects as go
import plotly.express as px
import nltk
from nltk.corpus import stopwords


Para este trabajo creé embeddings de las dos obras de **Jose Hernández**:

- **El Gaucho Martin Fierro**
- **La vuelta de Martin Fierro**

Estos archivos fueron extraidos de la pagina propuesta textos.info. Descargue archivos ePub (.epub) y luego los transforme a txt en una herramienta de conversión online. Luego eliminé manualmente cualquier texto introductorio, prólogos o índices para quedarme únicamente con los versos de la obra.

Para la Tokeneización utilicé la libreria NLTK, en vez de tensor flow, porque tiene la posibilidad de descargar stop words en español. 




In [2]:

try:
    # 'punkt' es para el tokenizador (word_tokenize)
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')

try:
    # 'stopwords' es para la lista de filtrado
    nltk.data.find('corpora/stopwords')
except LookupError:
    nltk.download('stopwords')

try:
    # 'punkt_tab' es una dependencia específica de word_tokenize para español
    nltk.data.find('tokenizers/punkt_tab')
except LookupError:
    nltk.download('punkt_tab')



print("--- Recursos de NLTK listos. ---")


# --- Obtener la lista de Stop Words en Español ---
stop_words_es = set(stopwords.words('spanish'))
print(f"Cargadas {len(stop_words_es)} stop words en español.")

# --- Cargar y combinar tus archivos .txt ---
file_names = [
    'Jose-Hernandez-El-Gaucho-Martin-Fierro.txt', 
    'Jose-Hernandez-La-Vuelta-de-Martin-Fierro.txt'
]

full_text = ""
for filename in file_names:
    print(f"Cargando {filename}...")
    with open(filename, 'r', encoding='utf-8') as f:
        full_text += f.read()
        full_text += "\n"

print(f"\n¡Corpus cargado! Total de caracteres: {len(full_text)}")

# --- Separar el texto en LÍNEAS (VERSOS) ---
lineas_o_versos = full_text.split('\n')
print(f"Total de líneas (versos) encontradas: {len(lineas_o_versos)}")


sentence_tokens = []

for linea in lineas_o_versos:
    if linea.strip(): # Ignorar líneas vacías
        
        # Tokenizamos con NLTK
        
        tokens_originales = nltk.word_tokenize(linea, language='spanish')
        
        # Filtramos, limpiamos y chequeamos stop words en un solo paso
        tokens_filtrados = []
        for palabra in tokens_originales:
            palabra_lower = palabra.lower()
            
            if (palabra_lower not in stop_words_es and 
                palabra.isalpha()): # .isalpha() filtra puntuación y números
                
                tokens_filtrados.append(palabra_lower)
        
        if tokens_filtrados: 
            sentence_tokens.append(tokens_filtrados)

# --- Verificar el resultado ---
print(f"Total de versos tokenizados y filtrados para Gensim: {len(sentence_tokens)}")
print("\nEjemplo de 2 versos filtrados (sin stop words):")
print(sentence_tokens[2:4])

--- Recursos de NLTK listos. ---
Cargadas 313 stop words en español.
Cargando Jose-Hernandez-El-Gaucho-Martin-Fierro.txt...
Cargando Jose-Hernandez-La-Vuelta-de-Martin-Fierro.txt...

¡Corpus cargado! Total de caracteres: 196014
Total de líneas (versos) encontradas: 14965
Total de versos tokenizados y filtrados para Gensim: 7218

Ejemplo de 2 versos filtrados (sin stop words):
[['compás', 'vigüela'], ['hombre', 'desvela']]


In [3]:
#Uso esta clase del ejemplo del notebook de la materia NLP

# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [4]:
# Crearmos el modelo generador de vectores con NUEVOS PARÁMETROS
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=3,    # BAJAMOS: para incluir más vocabulario gauchesco
                     window=5,       # SUBIMOS: para capturar el contexto del verso (incluso corto)
                     vector_size=100,  # BAJAMOS: 300 es mucho para un corpus tan chico. 100 es más robusto.
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

print("Modelo definido.")

Modelo definido.


In [5]:
w2v_model.build_vocab(sentence_tokens)

In [6]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=50,  # <-- SUBIMOS A 50 ÉPOCAS (para que aprenda más con un corpus chico)
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 147966.65625
Loss after epoch 1: 142907.25
Loss after epoch 2: 86670.5
Loss after epoch 3: 54403.28125
Loss after epoch 4: 45722.5
Loss after epoch 5: 43553.0625
Loss after epoch 6: 42533.0625
Loss after epoch 7: 42524.9375
Loss after epoch 8: 41818.6875
Loss after epoch 9: 42017.9375
Loss after epoch 10: 41735.0625
Loss after epoch 11: 41902.9375
Loss after epoch 12: 41806.3125
Loss after epoch 13: 41400.3125
Loss after epoch 14: 41299.25
Loss after epoch 15: 41184.3125
Loss after epoch 16: 41055.5625
Loss after epoch 17: 41144.1875
Loss after epoch 18: 39586.0625
Loss after epoch 19: 36243.375
Loss after epoch 20: 36516.875
Loss after epoch 21: 36572.625
Loss after epoch 22: 35986.0
Loss after epoch 23: 36247.625
Loss after epoch 24: 36444.625
Loss after epoch 25: 35939.875
Loss after epoch 26: 36175.75
Loss after epoch 27: 36029.5
Loss after epoch 28: 35632.25
Loss after epoch 29: 35347.25
Loss after epoch 30: 35196.125
Loss after epoch 31: 35025.5
Loss after epo

(517046, 840250)

In [7]:
print("Cantidad de words distintas en MI corpus (Martín Fierro):", len(w2v_model.wv.index_to_key))

Cantidad de words distintas en MI corpus (Martín Fierro): 1375


## Palabras que mas se relacionan con...

Observemos los algunos términos interesantes y buscamos términos más similares y menos similares.

In [8]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["guitarra"], topn=10)

[('tomó', 0.9800891280174255),
 ('irán', 0.9667337536811829),
 ('muchachos', 0.9604082703590393),
 ('prudencia', 0.9583792686462402),
 ('hice', 0.9577958583831787),
 ('estancia', 0.9523252248764038),
 ('duebla', 0.9484251141548157),
 ('martín', 0.9452843070030212),
 ('viera', 0.9419227242469788),
 ('pasar', 0.9297460913658142)]

pareciera que estas palabras no tienen mucho que ver con guitarra...

In [9]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["baile"], topn=10)

[('tuito', 0.9909536242485046),
 ('oveja', 0.9890865087509155),
 ('templar', 0.9886506795883179),
 ('oye', 0.9881430268287659),
 ('arisco', 0.9877519011497498),
 ('blanca', 0.9875532388687134),
 ('calor', 0.9874899983406067),
 ('prisión', 0.9874218702316284),
 ('compás', 0.9874205589294434),
 ('ventaja', 0.9872754216194153)]

Con baile tenemos mejores resultados. Salvo amargura las otras palabras parecen mas relacionadas.

In [10]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["amor"], topn=10)

[('facultá', 0.9825659394264221),
 ('consejos', 0.9778388142585754),
 ('alegre', 0.9755767583847046),
 ('aguas', 0.9745845198631287),
 ('defender', 0.9744588732719421),
 ('santos', 0.9740915894508362),
 ('flor', 0.9733414649963379),
 ('llama', 0.9727180004119873),
 ('tocó', 0.972646176815033),
 ('tabaco', 0.9720776677131653)]

In [11]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["cantar"], topn=10)

[('pongo', 0.8957429528236389),
 ('tutor', 0.8893966674804688),
 ('darles', 0.8848792910575867),
 ('canta', 0.8809892535209656),
 ('coronel', 0.8772412538528442),
 ('enteras', 0.8761800527572632),
 ('ladino', 0.8757993578910828),
 ('dejó', 0.8753611445426941),
 ('pedir', 0.8741534948348999),
 ('noches', 0.8740529417991638)]

In [12]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["desierto"], topn=10)

[('jesús', 0.9733541011810303),
 ('aguanta', 0.9698746800422668),
 ('mandaba', 0.9694339632987976),
 ('hermano', 0.9688454270362854),
 ('querido', 0.9680559039115906),
 ('dejando', 0.9678571224212646),
 ('destreza', 0.9678438305854797),
 ('dotor', 0.9677846431732178),
 ('amargo', 0.9674317240715027),
 ('crimen', 0.9665346145629883)]

In [13]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["caballo"], topn=10)

[('prendas', 0.9918239116668701),
 ('salieron', 0.9903663396835327),
 ('sable', 0.9903522729873657),
 ('lamento', 0.989538311958313),
 ('camina', 0.9894164204597473),
 ('toditos', 0.989342451095581),
 ('quitó', 0.9892719388008118),
 ('vuela', 0.9892050623893738),
 ('pan', 0.9891449809074402),
 ('malón', 0.9891298413276672)]

In [14]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["muerte"], topn=10)

[('pericón', 0.9668692946434021),
 ('nomás', 0.9662097096443176),
 ('fortuna', 0.9656972289085388),
 ('empezaba', 0.9648768901824951),
 ('sufriendo', 0.9646266102790833),
 ('juna', 0.9640106558799744),
 ('fiesta', 0.9634142518043518),
 ('tendido', 0.9631494283676147),
 ('sepoltura', 0.9627886414527893),
 ('consejos', 0.962566614151001)]

## Palabras que menos se relacionan con:

In [15]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["amor"], topn=10)

[('palos', 0.16626976430416107),
 ('gritó', 0.1262904852628708),
 ('segundo', 0.11171118170022964),
 ('naipe', 0.06864063441753387),
 ('invasión', 0.022730790078639984),
 ('dejamos', 0.007185106165707111),
 ('sierra', -0.060015980154275894),
 ('dejarlo', -0.12408607453107834),
 ('fierro', -0.5187598466873169),
 ('tal', -0.5282029509544373)]

In [16]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["desierto"], topn=10)

[('palos', 0.18847836554050446),
 ('dejamos', 0.10068265348672867),
 ('segundo', 0.09121239930391312),
 ('naipe', 0.06814339011907578),
 ('gritó', 0.02698598802089691),
 ('invasión', -0.0233039241284132),
 ('sierra', -0.10043489933013916),
 ('dejarlo', -0.16931293904781342),
 ('tal', -0.5109026432037354),
 ('vez', -0.5118721723556519)]

In [17]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["caballo"], topn=10)

[('palos', 0.2079288512468338),
 ('segundo', 0.127070352435112),
 ('gritó', 0.0832991749048233),
 ('naipe', 0.07071513682603836),
 ('dejamos', 0.05717218294739723),
 ('invasión', 0.047324392944574356),
 ('sierra', -0.09891247004270554),
 ('dejarlo', -0.14936867356300354),
 ('vez', -0.5432955026626587),
 ('tal', -0.5441628694534302)]

Para poder ver mejor las semejanzas de palabras, realizamos un gráfico de dos dimensiones. Para ello primero debemos realizar una disminucion de dimensionalidad.

In [18]:
def reduce_dimensions(model, num_dimensions = 2 ):

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [19]:
# Graficar los embedddings en 2D

vecs, labels = reduce_dimensions(w2v_model)

MAX_WORDS=800
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
fig.show()

## Encontrando familiaridad entre las palabras:

Hace ya varios años que leí Martin Fierro en la escuela primaria, y aunque me cueste un poco, trataré de encontrar similitudes en las palabras. ¿Por qué algunas palabras estan cerca de otras en el gráfico?


-[conmigo, perro, cimarron] como "compañía". el modelo parece haber encontrado es un cluster semántico de la "compañía en la soledad".

El gaucho, como lo describe el poema, es fundamentalmente un ser solitario.

En esa soledad, ¿cuáles son sus únicos compañeros leales?

- Su perro (un tema clásico en la literatura gauchesca, el compañero de trabajo y de vida).

- Su cimarrón (el mate que toma solo, "sentao junto al jogón" como dice el propio poema).

- la palabra "conmigo" refuerza ese eje: es la compañía que se define en relación a uno mismo, a la primera persona.


![conmigoperrocimarron.png](./conmigoperrocimarron.png)




de la misma imagen:
- [muerto, hambre, vicho] esta muerto de hambre. Cuando está muerto de hambre come algun bicho de por ahí como mulitas o peludos.



- [sufrir, miedo, terrible, duro, trance, penas]: sentimientos negativos del gaucho

![sufrirmiedoterrible.png](./sufrirmiedoterrible.png)

y de la misma imágen pulperia y comer. En la pulperia se podia comer.



- [puso, ruda, suerte] La ruda se usa para atraer la buena suerte principalmente como un amuleto protector contra las malas energías. En la misma imagen se encuentra dolor y pena.

![pusorudasuerte.png](./pusorudasuerte.png)




